In [0]:
dbutils.widgets.text("p_file_date","2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.functions import sum, desc, when, count,col,rank
from pyspark.sql.window import Window

In [0]:
race_result_list = spark.read.format("delta").load(f"{presentation_inc_folder_path}/races_result") \
                    .filter(f"file_date = '{v_file_date}'")\
                    .select("race_year") \
                    .distinct() \
                    .collect()

In [0]:
%python
race_year_list = []
for race_year in race_result_list:
    race_year_list.append(race_year.race_year)

In [0]:
race_result_df = spark.read.format("delta").load(f"{presentation_inc_folder_path}/races_result") \
.filter(col('race_year').isin(race_year_list))

In [0]:
constructor_standing_df = race_result_df\
                    .groupBy('race_year','team')\
                    .agg(sum('points').alias('points'),
                         count(when(col('position') == 1, True)).alias('wins'))\
                    .orderBy(desc('wins'))

In [0]:
constructor_rank_spec = Window.partitionBy('race_year').orderBy(desc('points'), desc('wins'))
final_constructor_standing_df = constructor_standing_df.withColumn('constructor_rank', rank().over(constructor_rank_spec))

In [0]:
#overwrite_partition(constructor_standing_df, "f1_presentation_incrementals", "constructor_standing", "race_year")

In [0]:
merge_condition =  "tgt.team = src.team AND tgt.race_year = src.race_year"
merge_delta_data(constructor_standing_df, "f1_presentation_incrementals", "constructor_standing", presentation_inc_folder_path, merge_condition, "race_year")

In [0]:
%sql
SELECT * FROM f1_presentation_incrementals.constructor_standing